<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/4_dataset-structure/gp-dataset-structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터셋 구조 분석

- 저번에는 공식 튜토리얼을 보고 진행했었다. `tf_example/` 하위의 데이터셋을 가지고 시각화를 진행했었는데, 아무래도 tf_example 파싱 부분이 마음에 안들어서 `scenario/` 데이터셋을 가지고 구조 분석 및 시각화를 다시 진행해보려고 한다. (본 문서에서는 구조분석만)

- 총 3개의 proto 파일을 분석할 예정이다. **sceanario.proto** 에서 나머지 두 proto를 import 해서 사용하기 때문.
  - [scenario.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/scenario.proto)
    ```python
    import "waymo_open_dataset/protos/compressed_lidar.proto";
    import "waymo_open_dataset/protos/map.proto";
    ```
  - [map.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/map.proto)
  - [compressed_lidar.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/compressed_lidar.proto)

- proto 파일 분석 전 알아야 할 것
  - proto 파일은 Protocol Buffers라는 구글에서 만든 직렬화 데이터 저장 방식.
  - message로 선언.
  - message 하위 요소들은 `{속성} {자료형} {이름}={넘버링};` 형태로 선언  
  - 속성의 종류
    - optional : 없어도 됨.
    - repeated : 리스트 형태로 구현됨
    - oneof : 하위 요소 중 하나 선택


## 환경세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# 구글 드라이브 위치. 마운트 잘 됐는지 확인.
GDRIVE_WOD_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed"
VALIDSET_PATH   = GDRIVE_WOD_PATH + "/scenario/validation"

# 마운트 확인
!ls -al {VALIDSET_PATH} | wc -l

Mounted at /content/drive
151


In [2]:
%%capture

!pip3 install --upgrade pip
!pip install waymo-open-dataset-tf-2-11-0==1.5.1

## [1] 데이터셋 로드

In [3]:
!ls -al {VALIDSET_PATH} | head

total 40266256
-rw------- 1 root root 273686917 Apr 14 03:30 validation.tfrecord-00000-of-00150
-rw------- 1 root root 272253722 Apr 14 03:30 validation.tfrecord-00001-of-00150
-rw------- 1 root root 281894961 Apr 14 03:30 validation.tfrecord-00002-of-00150
-rw------- 1 root root 283858404 Apr 14 03:30 validation.tfrecord-00003-of-00150
-rw------- 1 root root 293786123 Apr 14 03:30 validation.tfrecord-00004-of-00150
-rw------- 1 root root 275962185 Apr 14 03:30 validation.tfrecord-00005-of-00150
-rw------- 1 root root 262424772 Apr 14 03:29 validation.tfrecord-00006-of-00150
-rw------- 1 root root 263608876 Apr 14 03:29 validation.tfrecord-00007-of-00150
-rw------- 1 root root 287419235 Apr 14 03:30 validation.tfrecord-00008-of-00150


In [4]:
import tensorflow as tf
from waymo_open_dataset.protos import scenario_pb2

# TFRecordDataset 클래스를 사용하여 데이터를 읽어들임
dataset = tf.data.TFRecordDataset(VALIDSET_PATH + '/validation.tfrecord-00000-of-00150')

for data in dataset:
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(bytearray(data.numpy()))
  break

scenario.scenario_id

'19a486cd29abd7a7'

## [2] Scenario 1-depth 분석

- 앞서 scenario 인스턴스를 생성해서 데이터를 파싱했다. proto 파일에 작성된 구조는 다음과 같다.
  ```proto
  message Scenario {
    reserved 9;

    optional string scenario_id = 5;
    repeated double timestamps_seconds = 1;
    optional int32 current_time_index = 10;
    repeated Track tracks = 2;
    repeated DynamicMapState dynamic_map_states = 7;
    repeated MapFeature map_features = 8;
    optional int32 sdc_track_index = 6;
    repeated int32 objects_of_interest = 4;
    repeated RequiredPrediction tracks_to_predict = 11;
    repeated CompressedFrameLaserData compressed_frame_laser_data = 12;
  }
  ```
- string, double, int 등 단순한 자료형으로 선언되어있는 요소들을 먼저 본다.

In [ ]:
# optional string scenario_id = 5;

# 시나리오 고유 넘버
scenario.scenario_id

'19a486cd29abd7a7'

In [ ]:
# repeated double timestamps_seconds

# 본 시나리오에서 각 장면을 측정한 시각
# 총 9초. 과거(1초) + 미래(9초)
# 실제 testset에서는 앞 1초만 제공되어 뒤 8초를 예측해야 함.
scenario.timestamps_seconds

[0.0, 0.10004, 0.20005, 0.3, 0.40001, 0.49998, 0.59998, 0.69998, 0.79999, 0.89999, 0.99999, 1.09995, 1.19996, 1.29992, 1.39991, 1.49988, 1.59987, 1.69992, 1.79991, 1.89994, 1.99994, 2.09994, 2.19994, 2.29995, 2.39998, 2.49998, 2.59998, 2.69998, 2.79998, 2.89998, 2.99998, 3.10001, 3.20005, 3.30009, 3.40013, 3.50013, 3.60013, 3.70017, 3.80017, 3.90017, 4.00017, 4.10018, 4.20014, 4.30011, 4.40006, 4.50002, 4.59999, 4.69995, 4.79992, 4.89988, 4.99988, 5.09984, 5.19984, 5.29984, 5.39984, 5.49988, 5.59989, 5.69993, 5.79993, 5.89996, 6.0, 6.1, 6.2, 6.30003, 6.40003, 6.5, 6.6, 6.7, 6.8, 6.9, 6.99997, 7.09997, 7.19996, 7.29993, 7.39988, 7.49988, 7.59988, 7.69988, 7.79988, 7.89992, 7.99992, 8.09992, 8.19993, 8.29996, 8.39997, 8.49997, 8.59997, 8.69997, 8.79998, 8.89998, 8.99994]

In [ ]:
# optional int32 current_time_index

# 전체 관측 중 '현재'라고 판단하는 시점의 index.
# 과거 1초가 거의 끝난 0.99999초 부분이 현재임을 알 수 있음.
print(scenario.current_time_index)
print(scenario.timestamps_seconds[scenario.current_time_index])

10
0.99999


In [ ]:
# repeated Track tracks = 2;

# Map 내에 존재하는 Agent들의 상태정보를 가지고 있음.
# Track 구조에 대해서는 2depth에서 다시 언급
print(type(scenario.tracks))
print(type(scenario.tracks[0]))

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'>
<class 'waymo_open_dataset.protos.scenario_pb2.Track'>


In [ ]:
# repeated DynamicMapState dynamic_map_states = 7;

# 맵 정보 중 동적인 요소에 대한 정보를 담고 있음.
# DynamicMapState 구조에 대해서는 2depth에서 다시 언급
print(type(scenario.dynamic_map_states))
print(type(scenario.dynamic_map_states[0]))

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'>
<class 'waymo_open_dataset.protos.scenario_pb2.DynamicMapState'>


In [ ]:
# repeated MapFeature map_features = 8;

# 맵 정보에 대한 정보를 담고 있음(정적)
# MapFeature 구조에 대해서는 map.proto에서 다시 언급
print(type(scenario.map_features))
print(type(scenario.map_features[0]))

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'>
<class 'waymo_open_dataset.protos.map_pb2.MapFeature'>


In [ ]:
# optional int32 sdc_track_index

# sdc : Self-Driving Car
# 각 장면에는 장면을 측정하고 있는 agent 존재
# Tracks 내 여러 agent 중 sdc_track_index에 해당하는 agent가 측정용 객체.
scenario.sdc_track_index

10

In [ ]:
# repeated int32 objects_of_interest

# Track 내부에 obejct id 정보가 있는데, 관심있는 대상의 object id를 모아놓은 것.
# ('관심있다'에 대한 필요성 및 중요도는 아직 잘 모르겠음)
scenario.objects_of_interest

[]

In [ ]:
# repeated RequiredPrediction tracks_to_predict

# 예측해야 할 track에 대한 정보를 담고있음.
# 이 외의 track은 예측 시 무시해도 됨.
scenario.tracks_to_predict

[track_index: 2
difficulty: LEVEL_1
]

In [ ]:
# repeated CompressedFrameLaserData compressed_frame_laser_data

# 라이다 데이터가 담기는 공간이라고 함.
# 근데 제공된 데이터셋에는 라이다 데이터셋이 따로 제공됨.
# (안쓰는건가,,?)
scenario.compressed_frame_laser_data

[]

## [3] Scenario 2-depth 분석

- scenario 메세지 내부에 기본 자료형으로 선언되지 않은 요소들이 있었음.
  - repeated **Track tracks** = 2;
  - repeated **DynamicMapState dynamic_map_states** = 7;
  - repeated **MapFeature map_features** = 8;
    - MapFeature는 map.proto에 선언되어 있으므로 추후 다시 언급
  - repeated **RequiredPrediction tracks_to_predict**
  - repeated **CompressedFrameLaserData compressed_frame_laser_data**
    - CompressedFrameLaserData는 compressed_lidar.proto에 선언되어 있으므로 추후 다시 언급
- 이 요소들을 한단계 더 나아가 분석해보고자 함.

### [3-1] repeated Track tracks = 2;

```proto
message Track {
  enum ObjectType {
    TYPE_UNSET = 0;  // This is an invalid state that indicates an error.
    TYPE_VEHICLE = 1;
    TYPE_PEDESTRIAN = 2;
    TYPE_CYCLIST = 3;
    TYPE_OTHER = 4;
  }

  optional int32 id = 1;
  optional ObjectType object_type = 2;
  repeated ObjectState states = 3;
}
```

In [ ]:
# repeated Track tracks = 2;

# Map 내에 존재하는 Agent들의 상태정보를 가지고 있음.
print(len(scenario.tracks))

track = scenario.tracks[0]
print(f'* Track id   : {track.id}')
print(f'* Track type : {track.object_type}') # TYPE_VEHICLE
print(f'      -> {track.object_type == track.TYPE_VEHICLE}')
print(type(track.states))
print(type(track.states[0])) # ObjectState에 대해서는 3depth에서 다룰 예정

11
* Track id   : 368
* Track type : 1
      -> True
<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'>
<class 'waymo_open_dataset.protos.scenario_pb2.ObjectState'>


### [3-2] repeated DynamicMapState dynamic_map_states = 7;

```proto
message DynamicMapState {
  // The traffic signal states for all observed signals at this time step.
  repeated TrafficSignalLaneState lane_states = 1;
}
```

In [ ]:
# repeated DynamicMapState dynamic_map_states = 7;

# 맵 정보 중 동적인 요소에 대한 정보를 담고 있음.
print(f'* dynamic map stats length: {len(scenario.dynamic_map_states)}')

dynamic_lane_states = scenario.dynamic_map_states[0].lane_states
print(f'* dynamic lane state length : {len(dynamic_lane_states)}')
dynamic_lane_states[0] # TrafficSignalLaneState에 대해서는 map.proto 에서 다시 언급

* dynamic map stats length: 91
* dynamic lane state length : 4


lane: 120
state: LANE_STATE_GO
stop_point {
  x: 8391.668352965295
  y: 7316.195065592189
  z: -14.848865196389456
}

### [3-3] repeated RequiredPrediction tracks_to_predict

```
message RequiredPrediction {
  enum DifficultyLevel {
    NONE = 0;
    LEVEL_1 = 1;
    LEVEL_2 = 2;
  }
  optional int32 track_index = 1;
  optional DifficultyLevel difficulty = 2;
}
```

In [ ]:
# repeated RequiredPrediction tracks_to_predict

# 예측해야 할 track에 대한 정보를 담고있음.
# 이 외의 track은 예측 시 무시해도 됨.
print(f'* TTP length  : {len(scenario.tracks_to_predict)}')

ttp = scenario.tracks_to_predict[0]
print(f'    * Track Index : {ttp.track_index}')
print(f'    * Difficulty  : {ttp.difficulty}')

print(ttp)

* TTP length  : 1
    * Track Index : 2
    * Difficulty  : 1
track_index: 2
difficulty: LEVEL_1



## [4] Scenario 3-depth 분석

`Scenario -> Track -> State` 부분을 자세히 살펴본다.

State는 ObjectState 라는 메세지로 선언되어 있으며, 다음과 같다.

```proto
message ObjectState {
  optional double center_x = 2;
  optional double center_y = 3;
  optional double center_z = 4;

  optional float length = 5;
  optional float width = 6;
  optional float height = 7;

  optional float heading = 8;

  optional float velocity_x = 9;
  optional float velocity_y = 10;
  
  optional bool valid = 11;
}
```

In [12]:
# repeated Track tracks = 2;

# Map 내에 존재하는 Agent들의 상태정보를 가지고 있음.
print(f'* 시나리오 ID : {scenario.scenario_id}')
print(f'  * 해당 시나리오 내 Track 개수 : {len(scenario.tracks)}')

track = scenario.tracks[0]
print(f'  * tracks[0]')
print(f'    * Track ID : {track.id}')
print(f'    * Track type : {track.object_type}') # TYPE_VEHICLE
print(f'    * 해당 Track의 state 개수 : {len(track.states)}')

state = track.states[0]
print(f'    * [0] State')

# map 상에서 bounding box의 중심점 x,y,z 좌표
print(f'      * center_x: {state.center_x}')
print(f'      * center_y: {state.center_y}')
print(f'      * center_z: {state.center_z}')

# bbox의 크기
print(f'      * length: {state.length} m')
print(f'      * width: {state.width} m')
print(f'      * height: {state.height} m')

# bbox의 yaw값
print(f'      * heading: {state.heading} yaw')

# bbox의 속도
print(f'      * velocity_x: {state.velocity_x} m/s')
print(f'      * velocity_y: {state.velocity_y} m/s')

# 데이터 유효성. 손실되거나 유효하지 않으면 False
print(f'      * valid: {state.valid}')

* 시나리오 ID : 19a486cd29abd7a7
  * 해당 시나리오 내 Track 개수 : 11
  * tracks[0]
    * Track ID : 368
    * Track type : 1
    * 해당 Track의 state 개수 : 91
    * [0] State
      * center_x: 8382.083984375
      * center_y: 7213.89013671875
      * center_z: -13.732300758361816
      * length: 4.414564609527588 m
      * width: 1.943734884262085 m
      * height: 1.4712934494018555 m
      * heading: -1.5578702688217163 yaw
      * velocity_x: 0.146484375 m/s
      * velocity_y: -19.8193359375 m/s
      * valid: True


## [5] Map 분석

앞서 **scenario.proto** 파일 내의 Scenario 메세지를 분석했다. 분석과정에서 map.proto에 존재하는 요소들이 있었는데, 다음과 같다.

1. scenario.map_features -> **repeated MapFeature**
2. scenario.dynamic_map_states -> **repeated DynamicMapState**

위 두 메세지를 중심적으로 살펴보려고 한다. 이번에는 depth 구분 없이 쭉 분석한다.

분석에 앞서 map.proto에서 공통적으로 사용되는 하나의 메세지를 살펴본다.
```proto
message MapPoint {
  // Position in meters. The origin is an arbitrary location.
  optional double x = 1;
  optional double y = 2;
  optional double z = 3;
}
```

x,y,z 좌표를 표현할 때는 MapPoint를 이용한다.

### [5-1] repeated DynamicMapState

```proto
message DynamicState {
  // The timestamp associated with the dynamic feature data.
  optional double timestamp_seconds = 1;

  // The set of traffic signal states for the associated time step.
  repeated TrafficSignalLaneState lane_states = 2;
}
```
- **DynamicMapState**는 동적인 맵 정보를 담고있다. 신호등 정보만 다루는 것으로 보인다.

In [26]:
dynamic_map_state = scenario.dynamic_map_states[0]
# print(dynamic_map_state.timestamp_seconds) # 속성이 없음. 필요없나?
print(f'* lane state length: {len(dynamic_map_state.lane_states)}')

* lane state length: 4


```proto
message TrafficSignalLaneState {
  // The ID for the MapFeature corresponding to the lane controlled by this
  // traffic signal state.
  optional int64 lane = 1;

  enum State {
    LANE_STATE_UNKNOWN = 0;

    // 화살표가 함께있는 신호등
    LANE_STATE_ARROW_STOP = 1;
    LANE_STATE_ARROW_CAUTION = 2;
    LANE_STATE_ARROW_GO = 3;

    // 일반적인 원형 신호등
    LANE_STATE_STOP = 4;
    LANE_STATE_CAUTION = 5;
    LANE_STATE_GO = 6;

    // 깜빡이는 신호등
    LANE_STATE_FLASHING_STOP = 7;
    LANE_STATE_FLASHING_CAUTION = 8;
  }

  // 신호등 상태
  optional State state = 2;

  // 객체가 멈춰야 하는 지점 = 정지선 위치
  optional MapPoint stop_point = 3;
}
```

In [31]:
lane_state = dynamic_map_state = scenario.dynamic_map_states[0].lane_states[0]
print(f'* 이 신호 정보가 영향을 미치는 lane의 ID : {lane_state.lane}')
print(f'* 신호상태 : {lane_state.state}') # LANE_STATE_GO
print(f'* 정지선 위치 : ')
print(lane_state.stop_point)

* 이 신호 정보가 영향을 미치는 lane의 ID : 120
* 신호상태 : 6
* 정지선 위치 : 
x: 8391.668352965295
y: 7316.195065592189
z: -14.848865196389456



### [5-2] repeated MapFeature

```proto
message MapFeature {
  optional int64 id = 1;

  oneof feature_data {
    LaneCenter lane = 3;
    RoadLine road_line = 4;
    RoadEdge road_edge = 5;
    StopSign stop_sign = 7;
    Crosswalk crosswalk = 8;
    SpeedBump speed_bump = 9;
    Driveway driveway = 10;
  }
}
```

In [60]:
map_feature = scenario.map_features[0]
print(map_feature.id)
print('---- feature_data ----')
print(map_feature.lane)
print(map_feature.road_line)
#print(map_feature.road_edge) # 여기에 데이터가 있음
print(map_feature.stop_sign)
print(map_feature.crosswalk)
print(map_feature.speed_bump)
print(map_feature.driveway)
print('---- end ----')

3
---- feature_data ----






---- end ----


- map_features에는 어떤 데이터인지 구분할 수 있는 type 정보가 없다. 어떻게 알 수 있을까?
  - HasField 라는 내부 메소드를 사용하면 된다.

In [61]:
if map_feature.HasField('lane'):
    print('lane')
elif map_feature.HasField('road_line'):
    print('road_line')
elif map_feature.HasField('road_edge'):
    print('road_edge')
elif map_feature.HasField('stop_sign'):
    print('stop_sign')
elif map_feature.HasField('crosswalk'):
    print('crosswalk')
elif map_feature.HasField('speed_bump'):
    print('speed_bump')
elif map_feature.HasField('driveway'):
    print('driveway')

road_edge


- 이제 각 요소 별로 message 선언부를 살펴보자

#### [5-2-1] LaneCenter lane

```proto
message LaneCenter {
  // The speed limit for this lane.
  optional double speed_limit_mph = 1;

  // Type of this lane.
  enum LaneType {
    TYPE_UNDEFINED = 0;
    TYPE_FREEWAY = 1;
    TYPE_SURFACE_STREET = 2;
    TYPE_BIKE_LANE = 3;
  }
  optional LaneType type = 2;

  // True if the lane interpolates between two other lanes.
  optional bool interpolating = 3;

  repeated MapPoint polyline = 8;
  
  repeated int64 entry_lanes = 9 [packed = true];
  repeated int64 exit_lanes = 10 [packed = true];
  repeated BoundarySegment left_boundaries = 13;
  repeated BoundarySegment right_boundaries = 14;
  repeated LaneNeighbor left_neighbors = 11;
  repeated LaneNeighbor right_neighbors = 12;
}
```

In [99]:
for map_feature in scenario.map_features:
  if map_feature.HasField('lane'):
    print(f'* Map Feature ID: {map_feature.id}')
    lane = map_feature.lane
    print(f'* 해당 도로의 속도제한: {lane.speed_limit_mph} mph')
    print(f'* 도로 종류: {lane.type}') # 2: TYPE_SURFACE_STREET
    print(f'* polyline의 점 개수: {len(map_feature.lane.polyline)}')
    print(f'* 이 도로로 들어올 수 있는 도로들의 ID: {lane.entry_lanes}')
    print(f'* 이 도로를 통해 나갈 수 있는 도로들의 ID: {lane.exit_lanes}')
    # BoundarySegment. 바로 다음에서 다시 설명
    print(f'* 좌측 도로 경계 개수: {len(lane.left_boundaries)}')
    print(f'* 우측 도로 경계 개수: {len(lane.right_boundaries)}')
    # LaneNeighbor. 바로 다음에서 다시 설명
    print(f'* 좌측 인접 차선 개수: {len(lane.left_neighbors)}')
    print(f'* 우측 인접 차선 개수: {len(lane.right_neighbors)}')
    break

* Map Feature ID: 73
* 해당 도로의 속도제한: 25.0 mph
* 도로 종류: 2
* polyline의 점 개수: 169
* 이 도로로 들어올 수 있는 도로들의 ID: []
* 이 도로를 통해 나갈 수 있는 도로들의 ID: [175, 173, 174]
* 좌측 도로 경계 개수: 0
* 우측 도로 경계 개수: 2
* 좌측 인접 차선 개수: 0
* 우측 인접 차선 개수: 0


- BoundarySegment: 도로 경계에 대한 정보

```proto
// A segment of a lane with a given adjacent boundary.
message BoundarySegment {
  // lane 경계의 시작과 끝지점 인덱스
  optional int32 lane_start_index = 1;
  optional int32 lane_end_index = 2;

  // 본 경계에 매칭되는 도로선 정보(RoadLine/RoadEdge)의 ID.
  optional int64 boundary_feature_id = 3;

  // 도로선의 종류. 만약 도로선이 아니라 RoadEdge에 해당한다면
  // TYPE_UNKNOWN으로 설정됨.
  optional RoadLine.RoadLineType boundary_type = 4;
}
```

- LaneNeighbor : 인접 차선에 대한 정보

```proto
message LaneNeighbor {
  // The feature ID of the neighbor lane.
  optional int64 feature_id = 1;

  // 현재차선 기준 이웃차선과 인접한 부분의 시작과 끝지점 인덱스
  optional int32 self_start_index = 2;
  optional int32 self_end_index = 3;

  // 이웃차선 기준 현재차선과 인접한 부분의 시작과 끝지점 인덱스
  optional int32 neighbor_start_index = 4;
  optional int32 neighbor_end_index = 5;

  // 현재차선과 이웃차선 사이 경계에 대한 정보
  repeated BoundarySegment boundaries = 6;
}
```

#### [5-2-2] RoadLine road_line

```proto
message RoadLine {
  // Type of this road line.
  enum RoadLineType {
    TYPE_UNKNOWN = 0;
    TYPE_BROKEN_SINGLE_WHITE = 1;
    TYPE_SOLID_SINGLE_WHITE = 2;
    TYPE_SOLID_DOUBLE_WHITE = 3;
    TYPE_BROKEN_SINGLE_YELLOW = 4;
    TYPE_BROKEN_DOUBLE_YELLOW = 5;
    TYPE_SOLID_SINGLE_YELLOW = 6;
    TYPE_SOLID_DOUBLE_YELLOW = 7;
    TYPE_PASSING_DOUBLE_YELLOW = 8;
  }

  // The type of the lane boundary.
  optional RoadLineType type = 1;

  // The polyline defining the road edge. A polyline is a list of points with
  // segments defined between consecutive points.
  repeated MapPoint polyline = 2;
}
```

In [83]:
for map_feature in scenario.map_features:
  if map_feature.HasField('road_line'):
    print(f'* Map Feature ID: {map_feature.id}')
    print(f'* 도로 선 종류: {map_feature.road_line.type}') # 2: TYPE_SOLID_SINGLE_WHITE
    print(f'* polyline의 점 개수: {len(map_feature.road_line.polyline)}')
    break

* Map Feature ID: 17
* 도로 선 종류: 2
* polyline의 점 개수: 138


#### [5-2-3] RoadEdge road_edge

```proto
message RoadEdge {
  // Type of this road edge.
  enum RoadEdgeType {
    TYPE_UNKNOWN = 0;
    // 도로와 도로가 아닌곳을 나누는 가장자리
    TYPE_ROAD_EDGE_BOUNDARY = 1;
    // 다른 교통과 분리되는 부분의 가장자리
    TYPE_ROAD_EDGE_MEDIAN = 2;
  }

  // The type of road edge.
  optional RoadEdgeType type = 1;

  // The polyline defining the road edge. A polyline is a list of points with
  // segments defined between consecutive points.
  repeated MapPoint polyline = 2;
}
```

In [80]:
for map_feature in scenario.map_features:
  if map_feature.HasField('road_edge'):
    print(f'* Map Feature ID: {map_feature.id}')
    print(f'* 도로 가장자리 종류: {map_feature.road_edge.type}') # 1: TYPE_ROAD_EDGE_BOUNDARY
    print(f'* polyline의 점 개수: {len(map_feature.road_edge.polyline)}')
    break

* Map Feature ID: 3
* 도로 가장자리 종류: 1
* polyline의 점 개수: 119


#### [5-2-4] StopSign stop_sign

```proto
message StopSign {
  // The IDs of lane features controlled by this stop sign.
  repeated int64 lane = 1;

  // The position of the stop sign.
  optional MapPoint position = 2;
}
```

In [74]:
for map_feature in scenario.map_features:
  if map_feature.HasField('stop_sign'):
    print(f'* Map Feature ID: {map_feature.id}')
    for lane_id in map_feature.stop_sign.lane:
      print(f'* 이 정지 표시에 영향을 받는 도로의 ID: {lane_id}')
    print('* 위치:')
    print(map_feature.stop_sign.position)
    break

* Map Feature ID: 189
* 이 정지 표시에 영향을 받는 도로의 ID: 173
* 이 정지 표시에 영향을 받는 도로의 ID: 174
* 이 정지 표시에 영향을 받는 도로의 ID: 175
* 위치:
x: 8372.941808625585
y: 7178.621260407005
z: -11.517769180857217



#### [5-2-5] Crosswalk crosswalk

```proto
message Crosswalk {
  // The polygon defining the outline of the crosswalk. The polygon is assumed
  // to be closed (i.e. a segment exists between the last point and the first
  // point).
  repeated MapPoint polygon = 1;
}
```

In [71]:
for map_feature in scenario.map_features:
  if map_feature.HasField('crosswalk'):
    print(f'* Map Feature ID: {map_feature.id}')
    print(map_feature.crosswalk)
    break

* Map Feature ID: 183
polygon {
  x: 8404.968667011328
  y: 7337.301398328979
  z: -14.858865196389447
}
polygon {
  x: 8404.94074472753
  y: 7340.828363021967
  z: -14.858865196389447
}
polygon {
  x: 8373.909781097123
  y: 7340.628723511088
  z: -14.858865196389447
}
polygon {
  x: 8373.9377033796
  y: 7337.1017588181
  z: -14.858865196389447
}



#### [5-2-6] SpeedBump speed_bump

```proto
message SpeedBump {
  // The polygon defining the outline of the speed bump. The polygon is assumed
  // to be closed (i.e. a segment exists between the last point and the first
  // point).
  repeated MapPoint polygon = 1;
}
```

In [70]:
for map_feature in scenario.map_features:
  if map_feature.HasField('speed_bump'):
    print(f'* Map Feature ID: {map_feature.id}')
    print(map_feature.speed_bump)
    break

* Map Feature ID: 186
polygon {
  x: 8462.497877030944
  y: 7336.414111613698
  z: -14.62386519638949
}
polygon {
  x: 8462.479262175519
  y: 7337.245942909422
  z: -14.62386519638949
}
polygon {
  x: 8440.541655312623
  y: 7336.813390635457
  z: -14.62386519638949
}
polygon {
  x: 8440.56027016805
  y: 7335.981559340521
  z: -14.62386519638949
}



#### [5-2-7] Driveway driveway

```proto
message Driveway {
  // The polygon defining the outline of the driveway region. The polygon is
  // assumed to be closed (i.e. a segment exists between the last point and the
  // first point).
  repeated MapPoint polygon = 1;
}
```

In [67]:
for map_feature in scenario.map_features:
  if map_feature.HasField('driveway'):
    print(f'* Map Feature ID: {map_feature.id}')
    print(map_feature.driveway)
    break

* Map Feature ID: 190
polygon {
  x: 8278.061891630727
  y: 7324.2139192803525
  z: -14.271789724691361
}
polygon {
  x: 8277.605827678755
  y: 7338.709965989203
  z: -14.271789724691361
}
polygon {
  x: 8273.715322939819
  y: 7338.6656016537145
  z: -14.271789724691361
}
polygon {
  x: 8273.882856637325
  y: 7324.2139192803525
  z: -14.271789724691361
}



# 기타

In [ ]:
# tutorial/4_dataset-structure/gp-dataset-structure.ipynb